## 1. Load Data

In [19]:
import json
import re
from PyPDF2 import PdfReader
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter

def extract_text_with_pypdf2(pdf_path):
    """Extract text from a PDF file using PyPDF2."""
    text = ""
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        text += page.extract_text()
    return text

def extract_text_with_fitz(pdf_path):
    """Extract text from a PDF file using fitz (PyMuPDF)."""
    text = ""
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):  # Start from 0 to include all pages
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

def preprocess_and_chunk_text(text):
    """
    Preprocess the text and split it into chunks with titles and contexts.
    - Titles include both the current chapter and article.
    - Contexts contain the text under each article.
    """
    # Define regex patterns for identifying chapters and articles
    chapter_pattern = r"(Chương\s+[IVXLCDM]+\s*[\n\r]*[A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ\s]+?)(?=\s*Điều|$)"
    article_pattern = r"(Điều\s+\d+[a-z]?\.\s*[A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][^\n]+)"
    
    # Split the text into sections based on chapters and articles
    sections = re.split(f"({chapter_pattern}|{article_pattern})", text, flags=re.DOTALL)
    
    # Initialize variables for processing
    current_chapter = None
    current_article = None
    chunks = []
    buffer = ""
    
    for section in sections:
        # Skip None or empty sections
        if section is None or not section.strip():
            continue
        
        # Check if the section is a chapter title
        chapter_match = re.match(chapter_pattern, section)
        if chapter_match:
            # If there's a previous article, save its content as a chunk
            if current_article and buffer.strip():
                chunk = {
                    "title": f"{current_article} {current_chapter}",
                    "context": buffer.strip()
                }
                chunks.append(chunk)
            
            # Update the current chapter
            current_chapter = section.strip()
            current_article = None  # Reset article when a new chapter starts
            buffer = ""  # Reset buffer for new chapter
            continue
        
        # Check if the section is an article title
        article_match = re.match(article_pattern, section)
        if article_match:
            # If there's a previous article, save its content as a chunk
            if current_article and buffer.strip():
                chunk = {
                    "title": f"{current_article} {current_chapter}",
                    "context": buffer.strip()
                }
                chunks.append(chunk)
            
            # Update the current article
            current_article = section.strip()
            buffer = ""  # Reset buffer for new article
            continue
        
        # If it's neither a chapter nor an article, it's part of the current article's content
        if current_article:
            buffer += " " + section.strip()
    
    # Add the last chunk if there's any remaining content
    if current_article and buffer.strip():
        chunk = {
            "title": f"{current_article} {current_chapter}",
            "context": buffer.strip()
        }
        chunks.append(chunk)
    
    return chunks

def split_long_context(title, context, max_length=800):
    """
    Split long context into smaller chunks using RecursiveCharacterTextSplitter.
    Each smaller chunk retains the same title.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_length,
        chunk_overlap=300,  # Overlap to ensure continuity between chunks
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    sub_chunks = splitter.split_text(context)
    return [{"title": title, "context": sub_chunk.strip()} for sub_chunk in sub_chunks]

def process_pdf(pdf_path, output_json, extraction_method="pypdf2"):
    """Process the PDF and save the output as JSON."""
    # Step 1: Extract text from the PDF using the specified method
    if extraction_method == "pypdf2":
        raw_text = extract_text_with_pypdf2(pdf_path)
    elif extraction_method == "fitz":
        raw_text = extract_text_with_fitz(pdf_path)
    else:
        raise ValueError("Unsupported extraction method. Choose 'pypdf2' or 'fitz'.")
    
    # Debug: Print first 500 characters of extracted text
    print(f"Extracted text (first 500 chars): {raw_text[:500]}")
    
    # Step 2: Preprocess and chunk the text
    chunks = preprocess_and_chunk_text(raw_text)
    
    # Step 3: Split long contexts into smaller chunks
    final_chunks = []
    for chunk in chunks:
        title = chunk["title"]
        context = chunk["context"]
        if len(context) > 800:  # If context is too long, split it
            sub_chunks = split_long_context(title, context)
            final_chunks.extend(sub_chunks)
        else:
            final_chunks.append(chunk)
    
    # Debug: Print number of chunks created
    print(f"Number of chunks created: {len(final_chunks)}")
    print(final_chunks)
    
    # Step 4: Save the chunks to a JSON file
    save_to_json(final_chunks, output_json)

def save_to_json(data, output_file):
    """Save the processed data to a JSON file."""
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# Example usage
if __name__ == "__main__":
    pdf_path = "../Data_indexing/RAG_data.pdf"
    output_json = "output.json"
    
    # Choose the extraction method: "pypdf2" or "fitz"
    extraction_method = "fitz"  # Change to "pypdf2" if needed
    
    process_pdf(pdf_path, output_json, extraction_method)

Extracted text (first 500 chars):  
58 
CÔNG BÁO/Số 1215 + 1216/Ngày 27-11-2023 
  
CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM 
Độc lập - Tự do - Hạnh phúc 
 
LUẬT 
GIAO THÔNG ĐƯỜNG BỘ 
 
Luật Giao thông đường bộ số 23/2008/QH12 ngày 13 tháng 11 năm 2008 của 
Quốc hội, có hiệu lực kể từ ngày 01 tháng 7 năm 2009, được sửa đổi, bổ sung bởi: 
1. Luật số 35/2018/QH14 ngày 20 tháng 11 năm 2018 của Quốc hội sửa đổi, 
bổ sung một số điều của 37 luật có liên quan đến quy hoạch, có hiệu lực kể từ ngày 01 
tháng 01 năm 2019; 
2. Luật Phòng, chống
Number of chunks created: 151
[{'title': 'Điều 1. Phạm vi điều chỉnh Chương I \nNHỮNG QUY ĐỊNH CHUNG', 'context': 'Luật này quy định về quy tắc giao thông đường bộ; kết cấu hạ tầng giao thông \nđường bộ; phương tiện và người tham gia giao thông đường bộ; vận tải đường bộ \nvà quản lý nhà nước về giao thông đường bộ.'}, {'title': 'Điều 2. Đối tượng áp dụng Chương I \nNHỮNG QUY ĐỊNH CHUNG', 'context': 'Luật này áp dụng đối với tổ chức, cá nhân liên quan đến giao 

In [20]:
import json

with open('/home/khoa/MLE-k3/M1-project/Deploying-an-Agentic-RAG-Pipeline-from-scratch-to-GKE-using-Jenkins/Data_indexing/output.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

processed_documents = []
for item in json_data:
    combined_text = f"Trích dẫn ở: {item['title']} \n Nội dung như sau: {item['context']}"
    processed_documents.append(combined_text)


In [21]:
processed_documents

['Trích dẫn ở: Điều 1. Phạm vi điều chỉnh Chương I \nNHỮNG QUY ĐỊNH CHUNG \n Nội dung như sau: Luật này quy định về quy tắc giao thông đường bộ; kết cấu hạ tầng giao thông \nđường bộ; phương tiện và người tham gia giao thông đường bộ; vận tải đường bộ \nvà quản lý nhà nước về giao thông đường bộ.',
 'Trích dẫn ở: Điều 2. Đối tượng áp dụng Chương I \nNHỮNG QUY ĐỊNH CHUNG \n Nội dung như sau: Luật này áp dụng đối với tổ chức, cá nhân liên quan đến giao thông đường bộ \ntrên lãnh thổ nước Cộng hòa xã hội chủ nghĩa Việt Nam.',
 'Trích dẫn ở: Điều 3. Giải thích từ ngữ Chương I \nNHỮNG QUY ĐỊNH CHUNG \n Nội dung như sau: Trong Luật này, các từ ngữ dưới đây được hiểu như sau: \n1. Đường bộ gồm đường, cầu đường bộ, hầm đường bộ, bến phà đường bộ. \n2. Công trình đường bộ gồm đường bộ, nơi dừng xe, đỗ xe trên đường bộ, đèn \ntín hiệu, biển báo hiệu, vạch kẻ đường, cọc tiêu, rào chắn, đảo giao thông, dải phân \ncách, cột cây số, tường, kè, hệ thống thoát nước, trạm kiểm tra tải trọng xe, trạm \n

## 2. Ingestion

In [22]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import time
import weaviate

client = weaviate.Client("http://localhost:8085")
# Tokenize các câu
tokenizer_sent = [tokenize(sent) for sent in processed_documents]

model = SentenceTransformer('dangvantuan/vietnamese-embedding', device='cpu')

def vectorize_documents(documents):
    before = time.time()
    # Tính toán embeddings cho tất cả các câu đã tokenize
    embeddings = model.encode(documents)
    print(embeddings)
    
    after = time.time()
    print("Vectorized {} items in {}s".format(len(embeddings), after - before))
    
    return embeddings

def init_weaviate_schema(client):
    # a simple schema containing just a single class for our posts
    schema = {
        "classes": [{
            "class": "Document",
            "vectorizer": "none", # explicitly tell Weaviate not to vectorize anything, we are providing the vectors ourselves through our BERT model
            "properties": [{
                "name": "content",
                "dataType": ["text"],
            }]
        }]
    }

    # cleanup from previous runs
    client.schema.delete_all()

    client.schema.create(schema)

def import_documents_with_vectors(documents, vectors, client):
    if len(documents) != len(vectors):
        raise Exception("len of documents ({}) and vectors ({}) does not match".format(len(documents), len(vectors)))
        
    for i, document in enumerate(documents):
        try:
            client.data_object.create(
                data_object={"content": document},
                class_name='Document',
                vector=vectors[i]
            )
        except Exception as e:
            print(f"Error importing document {i}: {e}")

# Initialize Weaviate schema
init_weaviate_schema(client)

# Vectorize documents using the updated function
vectors = vectorize_documents(tokenizer_sent)

# Import documents along with their vectors into Weaviate
import_documents_with_vectors(processed_documents, vectors, client)


[[ 0.21265185 -0.55192715 -0.20609905 ...  0.5134595  -0.30940297
   0.6021905 ]
 [ 0.19525068 -0.6732101  -0.34741616 ...  0.3022192  -0.11117374
   0.7024409 ]
 [ 0.29016224 -0.47649378 -0.2617373  ...  0.35630128 -0.38773307
   0.4296254 ]
 ...
 [ 0.35419765 -0.14695631 -0.25566846 ...  0.10325216 -0.58149034
   0.2099498 ]
 [-0.09158062 -0.43477893  0.34777275 ...  0.14517453 -0.21137875
   0.23122507]
 [ 0.09480972 -0.23918341  0.5740878  ...  0.50751406 -0.22489655
   0.6811572 ]]
Vectorized 151 items in 20.866355895996094s


In [23]:
import weaviate

client_weaviate = weaviate.Client("http://localhost:8085")
schema = client_weaviate.schema.get()
print(schema)


{'classes': [{'class': 'Document', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'multiTenancyConfig': {'autoTenantActivation': False, 'autoTenantCreation': False, 'enabled': False}, 'properties': [{'dataType': ['text'], 'indexFilterable': True, 'indexRangeFilters': False, 'indexSearchable': True, 'name': 'content', 'tokenization': 'word'}], 'replicationConfig': {'asyncEnabled': False, 'factor': 1}, 'shardingConfig': {'actualCount': 1, 'actualVirtualCount': 128, 'desiredCount': 1, 'desiredVirtualCount': 128, 'function': 'murmur3', 'key': '_id', 'strategy': 'hash', 'virtualPerPhysical': 128}, 'vectorIndexConfig': {'bq': {'enabled': False}, 'cleanupIntervalSeconds': 300, 'distance': 'cosine', 'dynamicEfFactor': 8, 'dynamicEfMax': 500, 'dynamicEfMin': 100, 'ef': -1, 'efConstruction': 128, 'flatSearchCutoff': 40000, 'maxConnections': 32, 'pq': {'bitCompression': False, 'centroids': 